In [1]:
from transformers import BertTokenizer
import numpy as np
from tqdm import tqdm
import tensorflow as tf

/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-15 14:22:20.410925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734265340.468493   74172 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734265340.485880   74172 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 14:22:20.615300: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [2]:
# processed_sentences_dev = np.load('processed_sentences_no_conct_dev.npy', allow_pickle=True)
# words_to_entitied_dev=np.load('words_to_entities_dev.npy', allow_pickle=True)
processed_sentences_train = np.load('processed_sentences_no_conct_train.npy', allow_pickle=True)
words_to_entitied_train=np.load('words_to_entities_train.npy', allow_pickle=True)

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
def prepare_data(sentences, labels, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    with tf.device('/CPU:0'):
        for sentence in tqdm(sentences, desc="Tokenizing sentences"):
            encoding = tokenizer(sentence,
                                 truncation=True,
                                 padding='max_length',
                                 max_length=max_length,
                                 return_tensors="tf")  # Use "tf" to return TensorFlow tensors

            input_ids.append(encoding["input_ids"])  # TensorFlow tensor
            attention_masks.append(encoding["attention_mask"])  # TensorFlow tensor
    
    # Convert the list of tensors into a single tensor (batch_size, sequence_length)
    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    # Convert labels to a TensorFlow tensor as well
    labels = tf.convert_to_tensor(labels)

    return input_ids, attention_masks, labels

In [5]:
entities=np.load('final_negated_entites.npy', allow_pickle=True)
entities

array(['QUANTITY', 'STYLE', 'SIZE', 'TOPPING', 'NOT_VOLUME',
       'NOT_TOPPING', 'NOT_SIZE', 'NUMBER', 'NOT_STYLE', 'VOLUME',
       'CONTAINERTYPE', 'NOT_CONTAINERTYPE', 'NOT_NUMBER',
       'NOT_DRINKTYPE', 'DRINKTYPE', 'NOT_QUANTITY'], dtype='<U17')

In [6]:
entity_labels = [['B-'+entity, 'I-'+entity] for entity in entities]
entity_labels = [label for sublist in entity_labels for label in sublist]
np.save('entity_labels.npy', entity_labels)

In [7]:
entities_id = {e: i+1 for i, e in enumerate(entity_labels)}
entities_id['0']=0
entities_id['O']=0
entities_id


{'B-QUANTITY': 1,
 'I-QUANTITY': 2,
 'B-STYLE': 3,
 'I-STYLE': 4,
 'B-SIZE': 5,
 'I-SIZE': 6,
 'B-TOPPING': 7,
 'I-TOPPING': 8,
 'B-NOT_VOLUME': 9,
 'I-NOT_VOLUME': 10,
 'B-NOT_TOPPING': 11,
 'I-NOT_TOPPING': 12,
 'B-NOT_SIZE': 13,
 'I-NOT_SIZE': 14,
 'B-NUMBER': 15,
 'I-NUMBER': 16,
 'B-NOT_STYLE': 17,
 'I-NOT_STYLE': 18,
 'B-VOLUME': 19,
 'I-VOLUME': 20,
 'B-CONTAINERTYPE': 21,
 'I-CONTAINERTYPE': 22,
 'B-NOT_CONTAINERTYPE': 23,
 'I-NOT_CONTAINERTYPE': 24,
 'B-NOT_NUMBER': 25,
 'I-NOT_NUMBER': 26,
 'B-NOT_DRINKTYPE': 27,
 'I-NOT_DRINKTYPE': 28,
 'B-DRINKTYPE': 29,
 'I-DRINKTYPE': 30,
 'B-NOT_QUANTITY': 31,
 'I-NOT_QUANTITY': 32,
 '0': 0,
 'O': 0}

In [8]:
def padding_labels(labels, max_len):
    for i in range(len(labels)):
        if len(labels[i]) < max_len:
            labels[i] = labels[i] + ['0'] * (max_len - len(labels[i]))
        else:
            labels[i] = labels[i][:max_len]
    return labels

In [9]:
labels_train=[['0']+[entities_id[i] for i in tq]+['0'] for tq in words_to_entitied_train]
padded_labels_train = padding_labels(labels_train, 30)
padded_labels_train=np.array(padded_labels_train, dtype=np.int32)

In [10]:
labels_dev=[['0']+[entities_id[i] for i in tq]+['0'] for tq in words_to_entitied_dev]
padded_labels_dev = padding_labels(labels_dev, 30)
padded_labels_dev=np.array(padded_labels_dev, dtype=np.int32)

In [13]:
input_ids_train, attention_masks_train, padded_labels_train = prepare_data(processed_sentences_train[:1000000], padded_labels_train[:1000000], tokenizer, 30)

Tokenizing sentences: 100%|██████████| 1000000/1000000 [04:38<00:00, 3593.54it/s]


In [14]:
np.save('input_ids_train_position1.npy', input_ids_train)
np.save('attention_masks_train_position1.npy', attention_masks_train)
np.save('padded_labels_train_position1.npy', padded_labels_train)

In [10]:
input_ids_train2, attention_masks_train2, padded_labels_train2 = prepare_data(processed_sentences_train[1000000:2000000], padded_labels_train[1000000:2000000], tokenizer, 30)

I0000 00:00:1734213454.995825   45598 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 1000000/1000000 [05:04<00:00, 3284.11it/s]


In [11]:
np.save('input_ids_train_position2.npy', input_ids_train2)
np.save('attention_masks_train_position2.npy', attention_masks_train2)
np.save('padded_labels_train_position2.npy', padded_labels_train2)

In [10]:
input_ids_train3, attention_masks_train3, padded_labels_train3 = prepare_data(processed_sentences_train[2000000:], padded_labels_train[2000000:], tokenizer, 30)

I0000 00:00:1734214039.046564   51132 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 456446/456446 [02:13<00:00, 3411.58it/s]


In [11]:
np.save('input_ids_train_position3.npy', input_ids_train3)
np.save('attention_masks_train_position3.npy', attention_masks_train3)
np.save('padded_labels_train_position3.npy', padded_labels_train3)

In [11]:
input_ids_dev, attention_masks_dev, padded_labels_dev = prepare_data(processed_sentences_dev, padded_labels_dev, tokenizer, 30)

I0000 00:00:1734212110.229057   33534 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 348/348 [00:00<00:00, 3049.25it/s]


In [12]:
np.save('input_ids_dev_position.npy', input_ids_dev)
np.save('attention_masks_dev_position.npy', attention_masks_dev)
np.save('padded_labels_dev_position.npy', padded_labels_dev)

In [3]:
input_ids_train_position=np.concat([np.load('input_ids_train_position1.npy'), np.load('input_ids_train_position2.npy'),np.load('input_ids_train_position3.npy')], axis=0)
attention_masks_train_position=np.concat([np.load('attention_masks_train_position1.npy'), np.load('attention_masks_train_position2.npy'),np.load('attention_masks_train_position3.npy')], axis=0)
padded_labels_train_position=np.concat([np.load('padded_labels_train_position1.npy'), np.load('padded_labels_train_position2.npy'),np.load('padded_labels_train_position3.npy')], axis=0)

np.save('input_ids_train_position.npy', input_ids_train_position)
np.save('attention_masks_train_position.npy', attention_masks_train_position)
np.save('padded_labels_train_position.npy', padded_labels_train_position)